In [1]:
from tqdm import tqdm
import json
import requests
import geopandas as gpd
import geocoder
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, Polygon
from func import *

In [2]:
specified_order = [
    '北投區', '士林區', '內湖區', '南港區',
    '松山區', '信義區', '中山區', '大同區',
    '中正區', '萬華區', '大安區', '文山區'
]

In [3]:
breastfeeding_original = get_datataipei_api('4350117f-8697-4903-8a49-a7a59f30eeb1')
breastfeeding_copy = breastfeeding_original.copy()
breastfeeding_copy

,_id,_importdate,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,優良哺集乳室認證效期,輪椅使用,貼心小提醒
0,1,"{'date': '2023-03-22 11:30:34.622182', 'timezo...",松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,無,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
1,2,"{'date': '2023-03-22 11:30:34.631819', 'timezo...",松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,112年8月,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...
2,3,"{'date': '2023-03-22 11:30:34.633594', 'timezo...",松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,無,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主
3,4,"{'date': '2023-03-22 11:30:34.635314', 'timezo...",松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主
4,5,"{'date': '2023-03-22 11:30:34.637038', 'timezo...",松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,534,"{'date': '2023-03-22 11:30:35.447849', 'timezo...",北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
534,535,"{'date': '2023-03-22 11:30:35.449273', 'timezo...",北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
535,536,"{'date': '2023-03-22 11:30:35.450817', 'timezo...",北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
536,537,"{'date': '2023-03-22 11:30:35.452159', 'timezo...",北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主


In [6]:
breastfeeding_chart = breastfeeding_copy["行政區"]
breastfeeding_chart

0      松山區
1      松山區
2      松山區
3      松山區
4      松山區
      ... 
533    北投區
534    北投區
535    北投區
536    北投區
537    北投區
Name: 行政區, Length: 538, dtype: object

In [10]:
breastfeeding_count = breastfeeding_chart.value_counts()

In [11]:
result = {
  "data": [
    {
      "name": "breastfeeding_room_count",
      "data": [
        { "x": dist, "y": int(breastfeeding_count[dist]) }
        for dist in specified_order
      ]
    }
  ]
}
result

{'data': [{'name': 'breastfeeding_room_count',
   'data': [{'x': '北投區', 'y': 41},
    {'x': '士林區', 'y': 47},
    {'x': '內湖區', 'y': 24},
    {'x': '南港區', 'y': 34},
    {'x': '松山區', 'y': 47},
    {'x': '信義區', 'y': 53},
    {'x': '中山區', 'y': 46},
    {'x': '大同區', 'y': 34},
    {'x': '中正區', 'y': 87},
    {'x': '萬華區', 'y': 36},
    {'x': '大安區', 'y': 41},
    {'x': '文山區', 'y': 48}]}]}

In [13]:
with open('4003.json', "w", encoding="utf-8") as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)